In [5]:
# imports
import javalang
import os
from app.database import Database
from app.classes import IterationStatus, BatchAttemptStatus, TestStatus, AttemptStatus,Dataset
from typing import List
from app.constants import CONFIG
import pickle
import xml.etree.ElementTree as ET
from app.utils import etree_to_dict
from tqdm import tqdm

In [29]:
# constants
PROJECT_PATH = r"D:\IDEA_Projects\TestJavaCode"
CLOVER_XML_PATH = fr"{PROJECT_PATH}\target\site\clover\clover.xml"
# CLOVER_XML_PATH = r"C:\Users\tianfy\Desktop\毕设\数据集\evo-cleaned\gson.xml"
DATASET_NAME = r"gson"

In [30]:
# read dataset
with open(f"{CONFIG.dataset_folder}/{DATASET_NAME}.pkl", "rb") as f:
    raw_dataset = pickle.load(f)
    dataset = Dataset(**raw_dataset)

In [4]:
# read result
db = Database()
all_dataset = db.get_all()
for b in all_dataset:
    print(b.dataset, b.doc_id)

read time: 0.0449981689453125
decompress time: 3.188593864440918
loads time: 3.391084671020508
assembling time: 2.979224920272827
lang_1_fixed 47
commons-csv 58
commons-cli 61
gson 66
jfreechart 82
lang_1_fixed 89
gson 98
commons-csv 99
commons-cli 100
jfreechart 101
jfreechart154 123
jfreechart154 126
lang_1_fixed 127
gson 128
jfreechart154 130


In [5]:
batch_li = [x for x in all_dataset if x.doc_id in [47]]
attempts: List[AttemptStatus] = []
for bas in batch_li:
    attempts += bas.attempts
print("focal method: ", len(attempts))
llm_method_coverage = {}
for a in attempts:
    if a.name in llm_method_coverage:
        print(a.name)
    llm_method_coverage[a.name] = (a.cover_value, a.idx)
print(len(llm_method_coverage))

focal method:  1728
Validate#notEmpty(T, String, Object...) : T
Validate#notEmpty(T) : T
Validate#notEmpty(T, String, Object...) : T
Validate#notEmpty(T) : T
Validate#validIndex(T, int, String, Object...) : T
Validate#validIndex(T, int) : T
1722


In [28]:
from app.constants import CoverageMode

total_branch_tupe = (0, 0)
total_line_tupe = (0, 0)
count = 0
# get_pass_attempts = lambda x:  [a for a in x if a.type == IterationStatus.Type.PASS]
# attempts = get_pass_attempts(attempts)
# attempt_signatures = set([(a.clazz, a.method) for a in attempts])

# data = {}
data2 = {}
# a,b = 0, 0
# for at in attempts:
#     _a ,_b = 0, 0
#     if at.iteration_status:
#         if at.iteration_status.test_status:
#             at.iteration_status.test_status.mode = CoverageMode.LINE_COVERAGE
#             _a,_b = at.iteration_status.test_status.cover_value
#             a += _a
#             b += _b
#
#     data[f"{'#'.join([at.clazz, at.method, str(at.idx)])}"]=  (_a, _b)
# print(a,b)

# print(len(attempt_signatures))
tree = ET.parse(CLOVER_XML_PATH)
root = tree.getroot()
root_dict = etree_to_dict(root)['coverage']
method_coverage = {} # {method_signature: (branch_rate, line_rate)}
for class_info in tqdm(dataset):
    for method_info in class_info:
        # if not (class_info.class_name, method_info.signature) in attempt_signatures:
        #     continue

        count += 1
        source_packages: list = root_dict['project'][0]['package']
        # 找到focal class & focal method
        source_package = None
        for package in source_packages:
            if class_info.package_reference == package['@name']:
                source_package = package
                break
        assert source_package is not None

        source_files = source_package['file']
        source_file = None
        for file in source_files:
            if f"{class_info.class_name}.java" == file['@name']:
                source_file = file
                break
        assert source_file is not None

        source_report = source_file
    
        report_lines = source_report['line']
        branches = [line for line in report_lines if line['@type'] == 'cond']
        lines = [line for line in report_lines if line['@type'] == 'stmt']
        
        # 当测试某一个方法时，筛选出这个方法的line/branch
        branches = [branch for branch in branches if
                    method_info.start <= int(branch["@num"]) <= method_info.end]
        lines = [line for line in lines if
                 method_info.start <= int(line["@num"]) <= method_info.end]
    
        uncovered_branch = []
        false_uncovered_branch = []
        true_uncovered_branch = []
        unreachable_branch = []
    
        for branch in branches:
            if branch['@truecount'] == '0' or branch['@falsecount'] == '0':
                uncovered_branch.append(branch)
            if branch['@truecount'] != '0' and branch['@falsecount'] == '0':
                false_uncovered_branch.append(branch)
            if branch['@truecount'] == '0' and branch['@falsecount'] != '0':
                true_uncovered_branch.append(branch)
            if branch['@truecount'] == '0' and branch['@falsecount'] == '0':
                unreachable_branch.append(branch)



        branch_num_uncovered = len(false_uncovered_branch) + len(true_uncovered_branch) + 2 * len(unreachable_branch)
        branch_num_total = 1 if len(branches) == 0 else len(branches) * 2
        branch_num_covered = branch_num_total - branch_num_uncovered



        uncovered_line = []
        for line in lines:
            if line['@count'] == '0':
                uncovered_line.append(line)

        line_num_uncovered = len(uncovered_line)
        line_num_total = 1 if len(lines) == 0 else len(lines)
        line_num_covered = line_num_total - line_num_uncovered


        if line_num_covered == 0:
            count -= 1
            continue
            branch_num_covered = 0

        branch_rate_tuple = (branch_num_covered, branch_num_total, branch_num_covered / branch_num_total)
        line_rate_tuple = (line_num_covered, line_num_total, line_num_covered / line_num_total)
        data2[f"{'#'.join([class_info.class_name, method_info.signature, str(count - 1)])}"] = (line_num_covered, line_num_total)
        # print(f"Class: {class_info.class_name}, Method: {method_info.name}")
        # print(f"Branch Coverage: {branch_rate_tuple}")
        # print(f"Line Coverage: {line_rate_tuple}")
        method_coverage[f"{class_info.class_name}#{method_info.signature}"] = (branch_rate_tuple, line_rate_tuple)
        total_branch_tupe = (total_branch_tupe[0] + branch_rate_tuple[0], total_branch_tupe[1] + branch_rate_tuple[1])
        total_line_tupe = (total_line_tupe[0] + line_rate_tuple[0], total_line_tupe[1] + line_rate_tuple[1])
print(count)
print(f"Total Branch Coverage:  {total_branch_tupe} {total_branch_tupe[0] / total_branch_tupe[1]}")
print(f"Total Line Coverage: {total_line_tupe} {total_line_tupe[0] / total_line_tupe[1]}")

20it [00:00, 4000.48it/s]

9
Total Branch Coverage:  (10, 13) 0.7692307692307693
Total Line Coverage: (18, 21) 0.8571428571428571


In [7]:
# compare data and data2
c = 0
for k, v in data.items():
    if k not in data2:
        c += 1
        continue
    print(k, v, data2[k])

print(c)

AnnotationUtils#hashCode(Annotation) : int#2 (19, 22) (6, 11)
AnnotationUtils#toString(Annotation) : String#3 (15, 18) (7, 9)
AnnotationUtils#isValidAnnotationMemberType(Class<>) : boolean#4 (10, 10) (4, 5)
ArrayUtils#toString(Object) : String#5 (2, 2) (0, 1)
ArrayUtils#toString(Object, String) : String#6 (6, 6) (0, 3)
ArrayUtils#hashCode(Object) : int#7 (2, 2) (0, 1)
ArrayUtils#isEquals(Object, Object) : boolean#8 (2, 2) (0, 1)
ArrayUtils#toArray(T...) : T[]#10 (2, 2) (0, 1)
ArrayUtils#clone(T[]) : T[]#11 (6, 6) (3, 3)
ArrayUtils#clone(long[]) : long[]#12 (6, 6) (2, 3)
ArrayUtils#clone(int[]) : int[]#13 (6, 6) (0, 3)
ArrayUtils#clone(short[]) : short[]#14 (6, 6) (0, 3)
ArrayUtils#clone(char[]) : char[]#15 (6, 6) (3, 3)
ArrayUtils#clone(byte[]) : byte[]#16 (6, 6) (0, 3)
ArrayUtils#clone(double[]) : double[]#17 (6, 6) (0, 3)
ArrayUtils#clone(float[]) : float[]#18 (6, 6) (0, 3)
ArrayUtils#clone(boolean[]) : boolean[]#19 (6, 6) (0, 3)
ArrayUtils#nullToEmpty(String[]) : String[]#21 (6, 6) 

In [18]:
def g():
    for k, v in method_coverage.items():
        cover_value, idx = llm_method_coverage[k]
        llm_v = cover_value[0] / cover_value[1]
        evo_v = v[1][0] / v[1][1]
        if llm_v > evo_v and cover_value[1] > 10:
            yield f"{idx} {k}: {llm_v} {evo_v} {cover_value} {v}"
_g = g()

In [19]:
r = list(_g)


In [10]:
# fp = r"D:\IDEA_Projects\TestJavaCode\src\test\java\org\apache\commons\lang3\text\StrBuilder_ESTest.java"
# # 将这个文件 按照 32299、69379行分割成三个文件
# with open(fp, "r", encoding="utf-8") as f:
#     lines = f.readlines()
#     with open(fp.replace(".java", "_1.java"), "w", encoding="utf-8") as f1:
#         f1.writelines(lines[:32299])
#     with open(fp.replace(".java", "_2.java"), "w", encoding="utf-8") as f2:
#         f2.writelines(lines[32299:69379])
#     with open(fp.replace(".java", "_3.java"), "w", encoding="utf-8") as f3:
#         f3.writelines(lines[69379:])


In [17]:
def comp(a: str, b: str):
    if a.isdigit() or b.isdigit():
        return (b.isdigit() - a.isdigit()) or 0
    return (b.islower() - a.islower()) if a.isalpha() and b.isalpha() else 0
from functools import cmp_to_key
arr = list("B16sdw32A")
arr.sort(key=cmp_to_key(comp))
print(arr)

['1', '6', '3', '2', 's', 'd', 'w', 'B', 'A']


In [110]:
import random as r
from collections import Counter
T1 = [1,1,0]
T2 = [1,0,0]
count_list = []
from IPython.display import clear_output,display
import time

Counter({1: 5193599, 2: 2598401})


KeyboardInterrupt: 

dict_values([0.6844444444444444, 0.31555555555555553])